# Google Trends insight in understanding what drives Norwegians into purchasing an electric vehicle
Authors: 
- 270226 - Max Chevron 
- 321733 - Shasha Jiang 
- 314205 - Jostein Leirgulen 

Date: 03.12.2020 

In [1]:
import pickle
import pandas as pd 
from dateutil import relativedelta as rdelta
from datetime import datetime, date
import time
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats import diagnostic
import statsmodels.formula.api as smf
from pytrends.request import TrendReq
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.regression.linear_model as lm
from sklearn.metrics import mean_absolute_error

## Step 1 - Data wrangling

### Step 1.1 - EV sales and market share

In [2]:
sales = pd.read_csv('data/EV_sales_norway.csv',sep=';')

In [3]:
sales['weekday'] = ''
pd.to_datetime(sales['weekday'])
for index, row in sales.iterrows():
    # minus 1 because the dataset week system goes from 1 to 53 instead of 0 to 52, day start from sunday 0 to macth google trend
    sales.weekday[index] = datetime.strptime('{} {} 0'.format(row.year, row.week-1), '%Y %W %w').strftime("%Y-%m-%d") 
sales.weekday = pd.to_datetime(sales.weekday, format='%Y-%m-%d')
#compute market share for electric cars
sales['ev_share'] = sales['electric']/(sales['other'] + sales['electric'])
sales

C:\Users\Jostein\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,year,week,other,electric,weekday,ev_share
0,2010,1,17,0,2010-01-03,0.000000
1,2010,2,2776,0,2010-01-10,0.000000
2,2010,3,1953,0,2010-01-17,0.000000
3,2010,4,2078,0,2010-01-24,0.000000
4,2010,5,2873,0,2010-01-31,0.000000
...,...,...,...,...,...,...
572,2020,44,1749,1869,2020-11-01,0.516584
573,2020,45,961,1291,2020-11-08,0.573268
574,2020,46,1174,1435,2020-11-15,0.550019
575,2020,47,1259,1591,2020-11-22,0.558246


### Step 1.2 - Google trend data import and wrangling

In [4]:
# Function goal:      Convert start and end date to a string of dates with specific format
def convert_to_date(satrt,end):
      start = satrt.strftime("%Y-%m-%d")
      end = end.strftime("%Y-%m-%d")
      dates = start + ' ' + end
      return dates

In [5]:
'''
Function goal:      Split timespan in years
Function inputs:    date_start
                    date_end
Function outputs:   years_expand = Array years
'''

def split_data_into_year(date_start, date_end):
    years_expand = []
    start_year = date_start
    while abs(rdelta.relativedelta(start_year,date_end).years) >0 :
      one_year = rdelta.relativedelta(weeks=+50)
      end_year = start_year + one_year
    
      years_expand += [convert_to_date(start_year,end_year)]

      #update start_year
      one_week = rdelta.relativedelta(weeks=+1)
      start_year = end_year + one_week

    years_expand += [convert_to_date(start_year,date_end)]
    return years_expand

In [6]:
'''
Function goal:      Import Google Trends data
Function inputs:    kw = Keyword list 
                    geo = Country abreviation to search
                    dates = Timespan to search
                    eps = maximum propensity difference in matching
Function outputs:   pytrends = full pytrends format dataset
                    interest_over_time_df = Dataframe containing the interest evolution for each keyword
'''

def build_GT_data(kw, geo, dates):

    while True:
        try:
            pytrends.build_payload(kw_list=kw, cat=0, timeframe=dates, geo=geo, gprop='')
            break
        except:    
            pass # Sometimes Google Trends rejects the request. Try again until it works ! 

    interest_over_time_df = pytrends.interest_over_time()

    return interest_over_time_df[kw]

In [7]:
'''
Function goal:      Normalise the weekly values. It finds the date and keyword with the max value in weekly. 
                    Looks for the value in the monthly dataset which is smaller and normalises all the weekly value to this one.
Function inputs:    data_weekly = 1 year of Google Trends data with weekly granularity
                    data_monthly = Whole deseired timespan of Google Trends data with monthly granularity
                    kw = keywords
Function outputs:   data_weekly = same as input but normalised 
'''
def normalise(data_weekly, data_monthly, kw):
    # 1) Convert weekly into monthly with mean()
    data = data_weekly.groupby(data_weekly.index.strftime("%Y-%m")).mean()

    # 2) Find row index and column name of max value (which is not 100 as we have done the mean over 4 weeks)
    yearly_max = data.max().max()
    max_column = data.max().idxmax()
    max_row = data[max_column].idxmax()

    # 3) For the whole timespan (with monhtly datapoints), the value at the weekly max is 'true_max' 
    true_max = data_monthly[max_column].loc[data_monthly.index > max_row].iloc[0]

    # 4) Normalise each weekly datapoints by this true_max/yearly_max in %
    data_weekly = data_weekly*(true_max/yearly_max)/100

    return data_weekly

In [8]:
'''
Function goal:      Split the timespan in years so that we can extract weekly datapoints. 
Function inputs:    data_weekly = 1 year of Google Trends data with weekly granularity
                    interest_monthly = Whole deseired timespan of Google Trends data with monthly granularity
                    kw = keywords
                    yrs = full timespan split into years 'start_date end_date' for each year as a list
                    geo = Country abreviation to search
Function outputs:   interest = full timespan interest with weekly datapoints
'''
def get_weekly_interest(interest_monthly, kw, yrs, geo):
    interest = pd.DataFrame()
    for yearly_dates in yrs:
        interest_weekly = build_GT_data(kw, geo, yearly_dates)
        
        #if the search is for very short timespan it will result in daily instead of weekly values. So we take the weekly mean in case this happens
        interest_weekly['date'] = pd.to_datetime(interest_weekly.index) - pd.to_timedelta(7, unit='d')
        interest_weekly = interest_weekly.groupby(pd.Grouper(key='date', freq='W-SUN')).mean()
        
        interest_weekly = normalise(interest_weekly, interest_monthly, kw)
        interest = pd.concat([interest, interest_weekly])
    #The max week might actually not be around the same date as the max month. Hence we need to normalise again here so that max value is 100
    interest = interest/interest.max().max()*100
    return interest

In [9]:
### Eco keywords: ["CO2 bil", "Bil utslipp", "Global oppvarming", "Bil miljø"]

In [30]:
UTC_offset = 60 # Minutes for Norway (UTC+1)
host_language = 'en_US'
# keywords related to category: economical 
kw_eco = ["Elbil bompenger", "Elbil avgift", "Elbil pris", "Elbil fordeler", "Elbil parkering"] # REMOVED:, "Bompenger", "Bompenger pris", "Parkering pris", ] 
# keywords related to category: EV models (top 10 EV models sold in Norway with Tesla models grouped together as 'Tesla')
kw_evm = ["Tesla", "Nissan Leaf","Volkswagen e-Golf", "Bmw I3", "Kia Soul"] # REMOVED:, "Renault Zoe", "Audi E-tron", "Hyundai Ioniq"] 
# keywords related to category: range anxiety 
kw_ran = ["Elbil lading", "Elbil ladestasjoner", "Ladestasjoner", "Hurtiglading", "Elbil rekkevidde"] # REMOVED:, "Tesla supercharger"] 
# keywords related to category: environement 
#kw_env = ["Karbonavtrykk", "Luftkvalitet", "CO2", "CO2 bil", "CO2 utslipp"] 
kw_env = ["CO2 bil", "Bil utslipp", "Global oppvarming", "Bil miljø", "Karbonavtrykk"]
# all keywords grouped
kw_all = kw_eco + kw_evm + kw_ran + kw_env
#date_start = date(2009,12,28) # Sales dataset Timespan
#date_end = date(2020,11,29)
date_start = date(2010,1,3)
date_end = date(2020,1,5)
dates = convert_to_date(date_start,date_end)
yrs = split_data_into_year(date_start, date_end) # Will be usefull to extract yearly datasets to have weekly instead of monthly datapoints
country_abbreviation = 'NO' #Norway abreviation
pytrends = TrendReq(hl=host_language, tz=UTC_offset)

In [31]:
# Step 1: get non granular over results for reference:
# Giving a too large timespan yields unfortunately only monthy results. This will however serve as our reference frame for normalisation
interest_monthly_kw_eco = build_GT_data(kw_eco, country_abbreviation, dates)
interest_monthly_kw_evm = build_GT_data(kw_evm, country_abbreviation, dates)
interest_monthly_kw_ran = build_GT_data(kw_ran, country_abbreviation, dates)
interest_monthly_kw_env = build_GT_data(kw_env, country_abbreviation, dates)

In [32]:
#Step 2: get weekly 
interest_kw_eco = get_weekly_interest(interest_monthly_kw_eco, kw_eco, yrs, country_abbreviation)
interest_kw_evm = get_weekly_interest(interest_monthly_kw_evm, kw_evm, yrs, country_abbreviation)

#NOTE: GOOGLE DOES WEIRD THINGS SOMETIMES. SO YOU MIGHT NEED TO RUN A LINE AGAIN INDIVIDUALLY IF YOU HAVE AN ERROR
#For me it seems like its mainly for 'ran' that it fails but after a while if you run this line individually it works
# When you do too many queries you get rejected by Google Trends. You need to pause before starting again to avoid errors. 60 sec is recommended by pytrends
# Doesn't seem to work though ...
#time.sleep(60) 

interest_kw_ran = get_weekly_interest(interest_monthly_kw_ran, kw_ran, yrs, country_abbreviation)
interest_kw_env = get_weekly_interest(interest_monthly_kw_env, kw_env, yrs, country_abbreviation)

### Step 1.3 - Key events definition

In [33]:
# Economical related dates which penalises combustion vehicles (in favor of EV)
dates_eco = {"Climate settlement announcement":"2012-04-24",
             "Kyoto agreement second term":"2013-05-31"}

# EV model key releas dates
# Unknown date of the month for dates_evm so set to 15
dates_evm = {"Tesla model 3 first release":"2019-07-15",
             "Nissan Leaf first release":"2010-12-15"}

# Range anxiety mitigation key dates
dates_ran = {"Tesla supercharging system start":"2013-07-01"}

# Rising environmental awarness key dates
dates_env = {"Paris agreement anouncement":"2015-01-04",
             "Paris agreement updated emission cuts":"2020-02-27"}

# All dates together
dates_all = dict(dates_eco)
dates_all.update(dates_evm)
dates_all.update(dates_ran)
dates_all.update(dates_env)

In [34]:
sales_base = sales.copy()

# convert rows to month or year where each row is 1 week
#month_delay = 4
year_delay = 52

#a fix - t-1 should not be 4, which would imply we are trying to use last years week, and four weeks past in order 
# to predict NOW
week_delay = 1

# BOTH LOG AND LINEAR COLUMNS HERE. REMOVE ONE OR THE OTHER WHEN DECIDED: 
sales_base['ev_share_tm1']=sales_base.ev_share.shift(week_delay)
sales_base['ev_share_tm12']=sales_base.ev_share.shift(year_delay)
#replace 0 to a samll number bigger than 1 so that log can be defined
#sales_base['log_ev_share'] = np.log(sales_base.ev_share.replace(0, 0.5))
#sales_base['log_ev_share_tm1']=sales_base.log_ev_share.shift(month_delay)
#sales_base['log_ev_share_tm12']=sales_base.log_ev_share.shift(year_delay)

### Modifications by JOSTEIN
sales_base['log_ev_share'] = np.log(sales_base.ev_share + 0.1)
sales_base['log_ev_share_tm1']=sales_base.log_ev_share.shift(week_delay)
sales_base['log_ev_share_tm12']=sales_base.log_ev_share.shift(year_delay)

### Dataframes for statsmodels

In [35]:
df_sales = sales_base.iloc[:528,:].drop_duplicates(subset='weekday').set_index('weekday')
df_inter_eco = interest_kw_eco.iloc[1:,:]
df_inter_evm = interest_kw_evm.iloc[1:,:]
df_inter_ran = interest_kw_ran.iloc[1:,:]
df_inter_env = interest_kw_env.iloc[1:,:]

In [36]:
df_inter = df_inter_eco.merge(df_inter_evm, left_index=True, right_index=True)
df_inter = df_inter.merge(df_inter_ran, left_index=True, right_index=True)
df_inter = df_inter.merge(df_inter_env, left_index=True, right_index=True)
df_inter

,Elbil bompenger,Elbil avgift,Elbil pris,Elbil fordeler,Elbil parkering,Tesla,Nissan Leaf,Volkswagen e-Golf,Bmw I3,Kia Soul,Elbil lading,Elbil ladestasjoner,Ladestasjoner,Hurtiglading,Elbil rekkevidde,CO2 bil,Bil utslipp,Global oppvarming,Bil miljø,Karbonavtrykk
date,,,,,,,,,,,,,,,,,,,,
2010-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,1.274256,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.214559,0.0,0.000000
2010-01-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.042146,0.0,0.000000
2010-01-17,0.000000,0.000000,0.000000,0.000000,0.000000,1.243917,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.616858,0.0,0.000000
2010-01-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.425287,0.0,0.000000
2010-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.394413,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.808429,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-01,4.046915,2.023457,3.974649,1.951191,10.189554,32.644628,2.479339,0.0,0.413223,0.000000,10.814528,2.926284,25.064260,5.470879,5.216420,0.000000,15.126525,37.049026,0.0,0.000000
2019-12-08,0.000000,0.000000,6.648503,0.000000,4.046915,27.272727,2.066116,0.0,0.000000,0.000000,13.740812,5.343649,19.338921,8.524393,8.397163,15.784200,7.453650,36.172126,0.0,0.000000
2019-12-15,4.408247,2.457055,8.599694,0.000000,10.695418,28.512397,2.479339,0.0,0.826446,0.413223,6.234257,0.000000,19.847840,0.000000,14.376961,7.234425,7.672875,37.925926,0.0,0.000000


## Pickle dataframes

In [49]:
#df_inter.to_pickle('pytrends_V2.pickle')
#df_sales.to_pickle('sales_V2.pickle')
